In [1]:
import nltk
import re
import pandas as pd
import numpy as np
import os
from pathlib import Path
from scipy import stats

In [79]:
#import data
train_data = "/Users/zxj/Desktop/study/semester3/MCS/humor/data/task-1/train.csv"
train_data_2="/Users/zxj/Desktop/study/semester3/MCS/humor/data/task-1/data/task-1/train_funlines.csv"
test_data="/Users/zxj/Desktop/study/semester3/MCS/humor/data/task-1/data/task-1/dev.csv"
test_eval_data = "/Users/zxj/Desktop/study/semester3/MCS/humor/data/task-1/data/task-1/test_eval.csv"

test_eval=pd.read_csv(test_eval_data)
test_eval_label=test_eval.meanGrade

#get orignial news
test_eval_news=test_eval.original
test_eval_funny_word=test_eval.edit
#get data with ascending sort by score
train1=pd.read_csv(train_data)
train2=pd.read_csv(train_data_2)
test=pd.read_csv(test_data)

#get label
train_label=list(train1.meanGrade)+list(train2.meanGrade)
test_label=test.meanGrade

#get orignial news
train_news=list(train1.original)+list(train2.original)
train_funny_word=list(train1.edit)+list(train2.edit)
test_news=test.original
test_funny_word=test.edit

In [70]:
counts=list(train_news)+list(test_news)+list(test_eval_news)

In [3]:
#extract edited word from news
def find_tag(news):
    p = re.compile(r'[<](.*?)/[>]', re.S)
    return re.findall(p, news)
print(train_news[0])

France is ‘ hunting down its citizens who joined <Isis/> ’ without trial in Iraq


Bag of word 

In [14]:
#do preprocess
from nltk.corpus import stopwords
stopwords=set(stopwords.words('english'))
tt=nltk.tokenize.regexp.WordPunctTokenizer()
lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()

def preprocess_news(sentence,funny_word):
    final=[]
    num=0
    for event in sentence:
        BOW={}
        ori=event
        
        #tokenize and remove Punctuation
        event=removePunctuation(event)
        event=tt.tokenize(event)
        for words in event:
            words=lemmatizer.lemmatize(words)
            if words not in stopwords:
                
                #generate Bag of words
                BOW[words.lower()]=BOW.get(words,0)+1
        
        #give more weight for funny word
        BOW[funny_word[num].lower()]=BOW.get(funny_word[num],0)+10
        final.append(BOW)
        num+=1
    return final

punctuation = '!,;:?"\'.\'/<>'
def removePunctuation(text):
    text = re.sub(r'[{}]+'.format(punctuation),'',text)
    return text.strip()


In [ ]:
train_news=preprocess_news(train_news,train_funny_word)
test_news=preprocess_news(test_news,test_funny_word)

N-gram

In [71]:
from collections import defaultdict
from collections import Counter


def convert_sentence(sentence):
    return ["<s>"] + [w.lower() for w in sentence] + ["</s>"]

def get_counts(sentences):
    bigram_counts = defaultdict(Counter)
    unigram_counts = Counter()
    start_count = 0  # "<s>" counts: need these for bigram probs

    # collect initial unigram statistics
    for sentence in sentences:
        sentence=removePunctuation(sentence)
        sentence=tt.tokenize(sentence)
        sentence = convert_sentence(sentence)
        for word in sentence: # from 1, so we don't generate the <s> token
            unigram_counts[word] += 1
        start_count += 1

    # collect bigram counts
    for sentence in sentences:
        sentence=removePunctuation(sentence)
        sentence=tt.tokenize(sentence)
        sentence = convert_sentence(sentence)
        # generate a list of bigrams
        bigram_list = zip(sentence[:-1], sentence[1:])
        # iterate over bigrams
        for bigram in bigram_list:
            first, second = bigram
            bigram_counts[first][second] += 1
            
    token_count = float(sum(unigram_counts.values()))
    return unigram_counts, bigram_counts, start_count, token_count

In [44]:
unigram_counts, bigram_counts, start_count, token_count=get_counts(counts)

In [66]:
"a"+"b"

'ab'

In [80]:
def preprocess_bigram(sentence,funny_word):
    final=[]
    num=0
    for event in sentence:
        BOW = defaultdict(Counter)
        ori=event
        
        #tokenize and remove Punctuation
        event=removePunctuation(event)
        event=tt.tokenize(event)
        event=convert_sentence(event)
        for num in range(len(event)-1):
                word1=event[num]
                word2=event[num+1]
                score=bigram_counts[word1][word2]/unigram_counts[word1]
                #generate Bag of words
                BOW[(word1,word2)]=score
        
        #give more weight for funny word
        final.append(BOW)
        num+=1
    return final

In [81]:
train_news=preprocess_bigram(train_news,train_funny_word)
test_news=preprocess_bigram(test_news,test_funny_word)
test_eval_news=preprocess_bigram(test_eval_news,test_eval_funny_word)

In [77]:
print(len(train_news))
print(len(test_news))
print(len(train_label))
print(len(test_label))
print(train_matrix)

17900
2419
17900
2419
  (0, 73916)	0.013756779143558282
  (0, 73294)	0.02032273313031601
  (0, 71846)	0.33816082685444426
  (0, 70921)	0.06700882744120325
  (0, 65874)	0.19002548748906983
  (0, 35813)	3.635228888685276
  (0, 35304)	0.2503567734409228
  (0, 35174)	0.12793049508229176
  (0, 34827)	0.37768611830496374
  (0, 34485)	1.7862453223963788
  (0, 33211)	0.05070463729431917
  (0, 31756)	3.2313145677202453
  (0, 26782)	0.33816082685444426
  (0, 21729)	0.2045635573318362
  (0, 16025)	1.5306226899727478
  (0, 2822)	0.013486641503341827
  (1, 70661)	0.21175482070468563
  (1, 65984)	1.5306226899727478
  (1, 62261)	0.062216247150370906
  (1, 61011)	0.5302754899175008
  (1, 60063)	0.31610685988567616
  (1, 54088)	0.13730935897874663
  (1, 47913)	0.31958056164266163
  (1, 39987)	2.2957795198486863
  (1, 33513)	0.018360663708552775
  :	:
  (17898, 69456)	0.7456879771662105
  (17898, 58911)	0.0019612131549752586
  (17898, 54856)	0.7136912123436567
  (17898, 54401)	0.6270459618287865
  (1789

In [84]:
#transform to vector
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
vectorizer=DictVectorizer()
transformer=TfidfTransformer(smooth_idf=False,norm=None)
train_matrix=vectorizer.fit_transform(train_news)
#use TF_IDF
train_matrix=transformer.fit_transform(train_matrix)

#transform test
test_matrix=vectorizer.transform(test_news)
test_matrix=transformer.transform(test_matrix)
print(test_matrix.shape)
print(len(train_label))
print(train_matrix.shape)

(2419, 74109)
17900
(17900, 74109)


In [85]:
#do prediction and evaluation matrix for later works
def prediction(model):
    #best=model.fit(train_matrix,train_label)
    pre=model.predict(test_matrix)
    rmse = np.sqrt(np.mean((test_label - pre)**2))
    psr=stats.pearsonr(np.array(pre),np.array(test_label))
    print("The person correlation coefficient is:")
    print(psr)
    print("The result of RMSE in the whole test data is:")
    print(rmse)
    pre=model.predict(test_matrix[:100])
    rmse = np.sqrt(np.mean((test_label[:100] - pre)**2))    
    print("The result of RMSE in the BEST 10% test data is:")
    print(rmse)
    pre=model.predict(test_matrix[:200])
    rmse = np.sqrt(np.mean((test_label[:200] - pre)**2))    
    print("The result of RMSE in the BEST 20% test data is:")
    print(rmse)
    pre=model.predict(test_matrix[:300])
    rmse = np.sqrt(np.mean((test_label[:300] - pre)**2))    
    print("The result of RMSE in the BEST 30% test data is:")
    print(rmse)

In [86]:
from sklearn.model_selection import ShuffleSplit,GridSearchCV
from sklearn import linear_model
from sklearn.metrics import make_scorer,mean_squared_error,r2_score

#use MSE as scorer
#scorer=make_scorer(mean_squared_error)
scorer=make_scorer(r2_score)
cv = ShuffleSplit(n_splits=5, test_size=0.1, random_state=0)

#LASSO Method (alpha from 0.1 to 100)
a=np.logspace(-2,4,10)
lasso=linear_model.Lasso(fit_intercept=True)
grid=GridSearchCV(lasso,param_grid={'alpha':a},cv=cv,scoring=scorer)
grid.fit(train_matrix,train_label)

print("The best parameters are {0.best_params_} with R2——score of {0.best_score_:.3g}".format(grid))
best=grid.best_estimator_ 
prediction(best)


The best parameters are {'alpha': 0.01} with R2——score of -0.00032
The person correlation coefficient is:
(-0.020239753226676954, 0.3197139532215622)
The result of RMSE in the whole test data is:
0.5990002561616723
The result of RMSE in the BEST 10% test data is:
0.5863093134034103
The result of RMSE in the BEST 20% test data is:
0.5821869173441703
The result of RMSE in the BEST 30% test data is:
0.5790585601019096


In [87]:
test_eval_matrix=vectorizer.transform(test_eval_news)
test_eval_matrix=transformer.transform(test_eval_matrix)

pre=grid.predict(test_eval_matrix)
rmse = np.sqrt(np.mean((test_eval_label - pre)**2))
print(rmse)
import pandas as pd
id=test_eval.id
dataframe = pd.DataFrame({'id':id,'pred':pre})
dataframe.to_csv("task-1-output.csv",index=False,sep=',')

0.5941466132512262


In [9]:
#ridge method 
ridge=linear_model.Ridge(solver='sparse_cg')
a=np.logspace(-2,5,10)
grid2=GridSearchCV(ridge,param_grid={'alpha':a},cv=cv,scoring=scorer)
grid2.fit(train_matrix,train_label)

print("The best parameters are {0.best_params_} with R2-score of {0.best_score_:.3g}".format(grid2))
prediction(grid2)

The best parameters are {'alpha': 16681.005372000593} with R2-score of 0.0921
The person correlation coefficient is:
(0.334558900664955, 6.669315156841847e-16)
The result of RMSE in the whole test data is:
0.5332356662450543
The result of RMSE in the BEST 10% test data is:
0.8166910431997259
The result of RMSE in the BEST 20% test data is:
0.6333492806158451
The result of RMSE in the BEST 30% test data is:
0.5262552755566311


In [ ]:
#elasticnet method
#alpha and ratio for L1
a=np.linspace(0.01,5,10)
b=np.linspace(0,1,5)
elastic=linear_model.ElasticNet()

grid3=GridSearchCV(elastic,param_grid={'alpha':a,'l1_ratio':b},cv=cv,scoring=make_scorer(mean_squared_error))
grid3.fit(train_matrix,train_label)

print("The best parameters are {0.best_params_} with R2-score of {0.best_score_:.3g}".format(grid3))
prediction(grid3)

In [ ]:
### from sklearn.svm import SVR
c_range=np.logspace(-2,5,10)
gamma=np.logspace(-5,1,10)
degree=[0,1,2,3,4]
svr=SVR(kernel='poly')

grid4=GridSearchCV(svr,param_grid={'epsilon':epsilon,'C':c_range,'degree':degree},cv=cv,scoring=scorer)
grid4.fit(train_matrix,train_label)
print("The best parameters are {0.best_params_} with R2-score of {0.best_score_:.3g}".format(grid4))
prediction(grid4)